## Dataset specification

Explain Dataset Principle, link, paradigm of interest
The (subject, session) that we're gonna grab

### I/O

#### Imports

In [42]:
import gzip, shutil
import time
import yaml
import glob
from nipype.interfaces.matlab import MatlabCommand
from nipype.interfaces.spm import SliceTiming, Realign, Coregister, Normalize12
MatlabCommand.set_default_matlab_cmd('/Applications/MATLAB_R2020a.app/bin/matlab')


#### Preferences Paradigm Dataset Variables

In [2]:
subject_id = ['01','04','05','06']
ses_id = ['19','15']
n_run_per_task = 2
tasks = ['Houses','Food','Faces','Paintings']
path = 'dataset/'
experiment_data = dict()

#### Recursively Extracting Anatomical and Functional Nii.gz archives 

In [3]:
#Allows processing by the spm matlab underlying engine

t1 = time.time()

for file in [f for f in glob.glob(path + "**/**.nii.gz", recursive=True)]:
    
    print('- Extracting file {}'.format(file[20:]))
    
    with gzip.open(file, 'r') as f_in, open(file[:-3], 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    
t2 = time.time()

print('Done Successfully in {} minutes !'.format((t2-t1)/60))


- Extracting file sub-01_ses-00_anat_sub-01_ses-00_T1w.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceHouses_dir-ap_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-ap_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-pa_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-pa_sbref.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-pa_bold.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_bold.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_bold.nii.gz
- Extracting file sub-01_ses-19_func_sub-01_ses-19_task-PreferenceHouses_dir-pa_sbref.nii.gz
- Extracting f

#### Experiment Data Files Paths Extraction

In [4]:
#Building a structred dict for any pair (subject,acquisition_type) 
#with acquisition_type being Anatomical or Functional Data

for sub in subject_id:
    
    experiment_data[sub] = {'func':None,'anat':None}
    experiment_data[sub]['func'] = [f[:-3] for f in glob.glob(path + "**/*func_sub-{}*.nii.gz".format(sub), recursive=True)]
    experiment_data[sub]['anat'] = [f[:-3] for f in glob.glob(path + "**/*anat_sub-{}*.nii.gz".format(sub), recursive=True)]
    

#### Dataset Structure

In [5]:
print(yaml.dump(experiment_data))

'01':
  anat:
  - dataset/sub-01/anat/sub-01_ses-00_anat_sub-01_ses-00_T1w.nii
  func:
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceHouses_dir-ap_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-ap_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-pa_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-pa_sbref.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-pa_bold.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_bold.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_bold.nii
  - dataset/sub-01/func/sub-01_ses-19_func_sub-01_ses-1

## fMRI Data Preprocessing Pipeline

#### Preprocessing Strategy

![Preprocesing Streams](preprocessing_streams.png)

The preprocessing strategy is articulated on the level of analysis performed. As shown on the diagram, the difference between the two approches stems from whether or not the *Spatial Normalization* step is included. This distinguishing feature revolves around two considerations:
    
   - Analytical consideration Willingness to not introduce too many unecessary transformations in the first level analysis pipeline as Spatial Normalization is intented to correct spatial structural variabilty amongs different subjects.
   - Practical consideration : Computationally intensive step that will drastically improve computation time if avoided when its benifit is not evident.
   
The structuration of each distinct processing stream is based upon Poldrack, R., Mumford, J., & Nichols, T. (2011). *Preprocessing fMRI data. In Handbook of Functional MRI Data Analysis* (pp. 34-52). Cambridge: Cambridge University Press.


Distorsion Correction is implicit. As we are in the context of posterior-anterior phase encoded data, SPM Realign module used in *Motion Correction* account for these benign distorsion (cf.https://en.wikibooks.org/wiki/Neuroimaging_Data_Processing/Field_map_correction). Moreover, we know from litterature on the neural correlates we want to assess that they're typically not lying in air/tissue interfaces.

Smoothing is directly performed in Python just before analysis as it allows better computation performance relative to the SPM interface.

Additionnaly, the script below include/exclude Slice Timing Correction to assess its potential impact on later statistical analysis.

#### fMRI Acquisition Specification

In [44]:
t_r = 2.0
voxel_sizes = [1.5,1.5,1.5]
n_slices = len([0.0, 1.0225, 0.0625, 1.085, 0.1275, 1.15, 0.1925, 1.2125, 0.255, 1.2775, 0.32, 1.34, 0.3825, 1.405, 0.4475, 1.47, 0.51, 1.5325, 0.575, 1.5975, 0.6375, 1.66, 0.7025, 1.725, 0.765, 1.7875, 0.83, 1.8525, 0.895, 1.915, 0.9575, 0.0, 1.0225, 0.0625, 1.085, 0.1275, 1.15, 0.1925, 1.2125, 0.255, 1.2775, 0.32, 1.34, 0.3825, 1.405, 0.4475, 1.47, 0.51, 1.5325, 0.575, 1.5975, 0.6375, 1.66, 0.7025, 1.725, 0.765, 1.7875, 0.83, 1.8525, 0.895, 1.915, 0.9575, 0.0, 1.0225, 0.0625, 1.085, 0.1275, 1.15, 0.1925, 1.2125, 0.255, 1.2775, 0.32, 1.34, 0.3825, 1.405, 0.4475, 1.47, 0.51, 1.5325, 0.575, 1.5975, 0.6375, 1.66, 0.7025, 1.725, 0.765, 1.7875, 0.83, 1.8525, 0.895, 1.915, 0.9575])
slice_ordering =  list(range(1,n_slices+1))
mni_template = 'TPM.nii'

### Preprocessing Computation 

#### First Level Analysis Pipeline

##### Slice Timing Correction Included

In [29]:
t1 = time.time()
print('* '*3 + 'fMRI Preprocessing Computation' + ' *'*3 + '\n' )

for sub in subject_id:
    
    print('\t Processing Subject {}'.format(sub))
    
    
    
    for func_data in experiment_data[sub]['func']:
        
        if func_data[-9:-4] == 'sbref':
            
            pass
        
        else:
            
            print('\t \t - {}'.format(func_data[20:]))

            a_func = func_data[:20] +'a'+ func_data[20:]
            r_func = a_func[:20] +'r'+ a_func[20:]
            
            
            # Slice Timing
            print('\t \t \t % Slice Timing Correction')
            st = SliceTiming()
            st.inputs.in_files = func_data
            st.inputs.num_slices = n_slices
            st.inputs.time_repetition = t_r
            st.inputs.time_acquisition = t_r - t_r/n_slices
            st.inputs.slice_order = slice_ordering
            st.inputs.ref_slice = 1
            st.run()

            # Realignement (Motion Correction)
            print('\t \t \t % Realignement Motion Correction')
            realign = Realign()
            realign.inputs.in_files = a_func
            realign.inputs.register_to_mean = True
            realign.run() 
            
            
            #Too Much Computationaly intensive (extensive amount of time spent)
            # Coregistration
            #print('\t \t \t % Coregistration')
            #coreg = Coregister()
            #coreg.inputs.target = experiment_data[sub]['anat'][0]
            #coreg.inputs.source = r_func
            #coreg.out_prefix ='w'
            #coreg.run() 

    

t2 = time.time()

print('Done Successfully in {} minutes !'.format((t2-t1)/60))




* * * fMRI Preprocessing Computation * * *

	 Processing Subject 01
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceHouses_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task

##### Slice Timing Correction Not Included

In [29]:
t1 = time.time()
print('* '*3 + 'fMRI Preprocessing Computation' + ' *'*3 + '\n' )

for sub in subject_id:
    
    print('\t Processing Subject {}'.format(sub))
    
    
    
    for func_data in experiment_data[sub]['func']:
        
        if func_data[-9:-4] == 'sbref':
            
            pass
        
        else:
            
            print('\t \t - {}'.format(func_data[20:]))

            a_func = func_data[:20] +'a'+ func_data[20:]
            r_func = a_func[:20] +'r'+ a_func[20:]
            
            
            # Slice Timing
            #print('\t \t \t % Slice Timing Correction')
            #st = SliceTiming()
            #st.inputs.in_files = func_data
            #st.inputs.num_slices = n_slices
            #st.inputs.time_repetition = t_r
            #st.inputs.time_acquisition = t_r - t_r/n_slices
            #st.inputs.slice_order = slice_ordering
            #st.inputs.ref_slice = 1
            #st.run()

            # Realignement (Motion Correction)
            print('\t \t \t % Realignement Motion Correction')
            realign = Realign()
            realign.inputs.in_files = a_func
            realign.inputs.register_to_mean = True
            realign.run() 
            
            
            #Too Much Computationaly intensive (extensive amount of time spent)
            # Coregistration
            #print('\t \t \t % Coregistration')
            #coreg = Coregister()
            #coreg.inputs.target = experiment_data[sub]['anat'][0]
            #coreg.inputs.source = r_func
            #coreg.out_prefix ='w'
            #coreg.run() 

    

t2 = time.time()

print('Done Successfully in {} minutes !'.format((t2-t1)/60))




* * * fMRI Preprocessing Computation * * *

	 Processing Subject 01
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFaces_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceHouses_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferencePaintings_dir-ap_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task-PreferenceFood_dir-pa_bold.nii
	 	 	 % Slice Timing Correction
	 	 	 % Realignement Motion Correction
	 	 - sub-01_ses-19_func_sub-01_ses-19_task

#### Second Level Analysis Pipeline Extension

In [ ]:
t1 = time.time()
print('* '*3 + 'fMRI Preprocessing Computation' + ' *'*3 + '\n' )

for sub in subject_id:
    
    print('\t Processing Subject {}'.format(sub))
    
    
    
    for func_data in experiment_data[sub]['func']:
        
        if func_data[-9:-4] == 'sbref':
            
            pass
        
        else:
            
            print('\t \t - {}'.format(func_data[20:]))

            ra_func = func_data[:20] +'ra'+ func_data[20:]
            
            
            
            # Slice Timing
            print('\t \t \t % Spatial Normalization')
            norm = Normalize12()
            norm.inputs.tpm = mni_template
            norm.inputs.write_voxel_sizes = voxel_sizes
            norm.inputs.image_to_align= ra_func
            norm.run() 


    

t2 = time.time()

print('Done Successfully in {} minutes !'.format((t2-t1)/60))
